In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import scipy.sparse as sps
import pickle
import seaborn
import matplotlib.pyplot as plt

from openomics.multiomics import MultiOmics
from openomics.database import *
from openomics.transcriptomics import *
from openomics.genomics import *

from moge.visualization.plot_data import matrix_heatmap, plot_coo_matrix
from moge.visualization.visualize_embedding import visualize_embedding, plot_bokeh_graph
from moge.network.omics_distance import *

# Import the TCGA cancer data

In [2]:
pd.set_option('mode.chained_assignment','raise')

In [3]:
cohort_folder = "/data/datasets/Bioinformatics_ExternalData/tcga-assembler/LUAD/"
cohort_name = "LUAD"
luad_data = MultiOmics(cohort_name)
luad_data.add_clinical_data(clinical_data="/data/datasets/Bioinformatics_ExternalData/tcga-assembler/LUAD/clinical/nationwidechildrens.org_clinical_patient.txt")

luad_data.add_omic(MessengerRNA(cohort_name, file_path=os.path.join(cohort_folder, "gene_exp", "geneExp.txt"), 
                                gene_index_by="gene_name",
                                columns="GeneSymbol|TCGA", genes_col_name="GeneSymbol"))

luad_data.add_omic(MicroRNA(cohort_name, file_path=os.path.join(cohort_folder, "mirna/", "miRNAExp__RPM.txt"), 
                            gene_index_by="gene_name",
                            columns="GeneSymbol|TCGA", genes_col_name="GeneSymbol"))

luad_data.add_omic(LncRNA(cohort_name, file_path=os.path.join(cohort_folder, "lncrna", "TCGA-rnaexpr.tsv"), 
                          gene_index_by="gene_id",
                          columns="Gene_ID|TCGA", genes_col_name="Gene_ID"))
luad_data.MicroRNA.expressions.index = luad_data.MicroRNA.expressions.index.str.replace("mir", "miR")
luad_data.MicroRNA.annotations.index = luad_data.MicroRNA.annotations.index.str.replace("mir", "miR")
luad_data.build_samples()

/opt/anaconda3/lib/python3.6/site-packages/openomics/transcriptomics.py:52: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.



MessengerRNA (576, 20472) , indexed by: gene_name
MicroRNA (494, 1870) , indexed by: gene_name
LncRNA (546, 12727) , indexed by: gene_id


In [4]:
rnacentral = RNAcentral(path="ftp://ftp.ebi.ac.uk/pub/databases/RNAcentral/current_release/",
                       file_resources={"rnacentral_rfam_annotations.tsv":"go_annotations/rnacentral_rfam_annotations.tsv.gz",
                                      "gencode.tsv":"id_mapping/database_mappings/gencode.tsv"},
                       )

mirbase = MirBase(path="ftp://mirbase.org/pub/mirbase/CURRENT/")
ensembl = EnsemblGenes()
gencode = GENCODE(path="ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_32/",
                  file_resources={"long_noncoding_RNAs.gtf": "gencode.v32.long_noncoding_RNAs.gtf.gz",
                                  "lncRNA_transcripts.fa": "gencode.v32.lncRNA_transcripts.fa.gz",
                                  "transcripts.fa": "gencode.v32.transcripts.fa.gz"},
                  import_sequences="shortest")
gtex = GTEx(path="https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/")
# interaction datasets
# lncbase = LncBase("/data/datasets/Bioinformatics_ExternalData/lncBase/", )

# targetscan = TargetScan("/data/datasets/Bioinformatics_ExternalData/TargetScan/", )
# mirtarbase = MiRTarBase("/data/datasets/Bioinformatics_ExternalData/miRTarBase/", target_index="Target Gene")

Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/RNAcentral/current_release/ go_annotations/rnacentral_rfam_annotations.tsv.gz
Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/RNAcentral/current_release/ id_mapping/database_mappings/gencode.tsv
RNAcentral: ['index', 'RNAcentral id', 'database', 'transcript_id', 'species', 'RNA type', 'gene_name', 'go_id', 'Rfams']
Fetching file from URL: ftp://mirbase.org/pub/mirbase/CURRENT/ aliases.txt.gz
Fetching file from URL: ftp://mirbase.org/pub/mirbase/CURRENT/ mature.fa.gz
Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/RNAcentral/current_release/id_mapping/database_mappings/ mirbase.tsv
MirBase: ['mirbase id', 'RNAcentral id', 'database', 'species', 'RNA type', 'gene name', 'gene_name']


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



EnsemblGenes ['gene_id', 'gene_name', 'transcript_id', 'transcript_name', 'chromosome_name', 'transcript_start', 'transcript_end', 'transcript_length', 'gene_biotype', 'transcript_biotype', 'Rfams', 'go_id']
Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_32/ gencode.v32.long_noncoding_RNAs.gtf.gz
Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_32/ gencode.v32.lncRNA_transcripts.fa.gz
Fetching file from URL: ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_32/ gencode.v32.transcripts.fa.gz


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'tag', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'ont']


GENCODE: ['index', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'tag', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'ont']
Fetching file from URL: https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/ GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz
Fetching file from URL: https://storage.googleapis.com/gtex_analysis_v8/annotations/ GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt
Fetching file from URL: https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/ GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct.gz
GTEx: ['gene_id', 'gene_name', 'Adipose - Subcutaneous', 'Adipose - Visceral (Omentum)', 'Adrenal Gland', 'Artery - Aorta', 'Artery - Coronary', 'Artery - Tibial', 'Bladder', 'Brain - Amygdala', 'Brain - Anterior cingulate cortex (BA24)', 'Brain - Caudat

In [5]:
disgenet = DisGeNet(path="https://www.disgenet.org/static/disgenet_ap1/files/downloads/", curated=True)
hmdd = HMDD(path="http://www.cuilab.cn/static/hmdd3/data/")
lncrnadisease = LncRNADisease(path="http://www.cuilab.cn/files/images/ldd/", species="Human")

Fetching file from URL: https://www.disgenet.org/static/disgenet_ap1/files/downloads/ curated_gene_disease_associations.tsv.gz
Fetching file from URL: https://www.disgenet.org/static/disgenet_ap1/files/downloads/ all_gene_disease_associations.tsv.gz
DisGeNet: ['index', 'gene_name', 'disease_associations', 'score']
Fetching file from URL: http://www.cuilab.cn/static/hmdd3/data/ alldata.txt
HMDD: ['index', 'category', 'gene_name', 'disease_associations', 'pmid', 'description']
Fetching file from URL: http://www.cuilab.cn/files/images/ldd/ data_v2017.txt
LncRNADisease: ['index', 'gene_name', 'disease_associations', 'Dysfunction type', 'Description', 'Chr', 'Start', 'End', 'Strand', 'Species', 'Alias', 'Sequence', 'Reference']


In [7]:
import difflib
miRNA_index = luad_data.MicroRNA.annotations.index.map(lambda x: difflib.get_close_matches(x, mirbase.df["gene_name"])[0])
luad_data.MicroRNA.annotations.index = miRNA_index
luad_data.MicroRNA.annotations.index & mirbase.df["gene_name"]

In [9]:
luad_data.LncRNA.annotate_genomics(gencode, index="gene_id", 
                                columns=['feature', 'start', 'end', 'strand', 'tag', 'havana_gene'])
luad_data.LncRNA.annotate_genomics(database=ensembl, index='gene_id', 
                                columns=['gene_name', 'transcript_id', 'transcript_name', 
                                         'chromosome_name', 'transcript_start', 'transcript_end', 'transcript_length',
                                         'Rfams', 'go_id', 'gene_biotype', 'transcript_biotype'])
luad_data.LncRNA.annotate_genomics(database=rnacentral, index='gene_name',
                                columns=['Rfams', 'go_id', 'gene_name'])
luad_data.LncRNA.annotate_sequences(gencode, index="gene_id", omic="LncRNA")
luad_data.LncRNA.annotate_diseases(lncrnadisease, index="gene_name", )
luad_data.LncRNA.annotations.info()

luad_data.MicroRNA.annotate_genomics(
    database=mirbase, 
    index="gene_name", 
    columns=['mirbase id', 'RNAcentral id', 'database'],)
luad_data.MicroRNA.annotate_genomics(
    database=rnacentral, 
    index="RNAcentral id",
    columns=['transcript_id', 'RNA type', 'go_id', 'Rfams']
)
luad_data.MicroRNA.annotate_genomics(
    database=ensembl, 
    index='gene_name',
    columns=['gene_name', 'transcript_id', 'transcript_name', 
             'chromosome_name', 'transcript_start', 'transcript_end', 'transcript_length',
             'Rfams', 'go_id', 'gene_biotype', 'transcript_biotype'])
luad_data.MicroRNA.annotate_sequences(mirbase, index="gene_name", omic="MicroRNA")
luad_data.MicroRNA.annotate_diseases(hmdd, index="gene_name", )
luad_data.MicroRNA.annotations.info()

luad_data.MessengerRNA.annotate_genomics(database=rnacentral, index="gene_name",
                                columns=['gene_name', 'transcript_id', 'RNA type', 'go_id', 'Rfams'])
luad_data.MessengerRNA.annotate_genomics(database=ensembl, index='gene_name',
                                columns=['gene_id', 'transcript_id', 'transcript_name', 
                                         'chromosome_name', 'transcript_start', 'transcript_end', 'transcript_length',
                                         'Rfams', 'go_id', 'gene_biotype', 'transcript_biotype'])
luad_data.MessengerRNA.annotate_sequences(gencode, index="gene_name", omic="MessengerRNA")
luad_data.MessengerRNA.annotate_diseases(disgenet, index="gene_name", )
luad_data.MessengerRNA.annotations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12727 entries, ENSG00000005206 to ENSGR0000270726
Data columns (total 19 columns):
gene_name               11848 non-null object
feature                 11586 non-null object
start                   11586 non-null object
end                     11586 non-null object
strand                  11586 non-null object
tag                     11586 non-null object
havana_gene             11586 non-null object
transcript_id           11848 non-null object
transcript_name         11848 non-null object
chromosome_name         11848 non-null object
transcript_start        11848 non-null object
transcript_end          11848 non-null object
transcript_length       11848 non-null object
Rfams                   150 non-null object
go_id                   216 non-null object
gene_biotype            11848 non-null object
transcript_biotype      11848 non-null object
Transcript sequence     11586 non-null object
disease_associations    13 non-null object
dtype

In [16]:
luad_data.MessengerRNA.annotate_expressions(database=gtex, index="gene_name")
# luad_data.LncRNA.annotate_expressions(database=gtex, index="gene_id")
# luad_data.MicroRNA.annotate_expressions(database=gtex, index="gene_name")

In [17]:
luad_data.MessengerRNA.get_annotation_expressions()

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),Brain - Caudate (basal ganglia),...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,3.109560,3.278810,1.405660,3.371560,7.405130,6.438560,2.171510,4.02207,3.555730,4.193790,...,2.340100,2.192790,2.804740,6.324840,1.350860,1.542190,4.536520,7.846930,5.491580,1.691720
A1BG-AS1,1.613390,1.959340,1.024820,2.109530,5.085090,3.536230,1.258740,1.00217,1.863250,1.452590,...,1.505240,1.257310,2.505390,5.426820,1.006890,0.640786,3.716960,6.283290,3.273600,1.280710
A1CF,0.007384,0.005619,0.018703,0.016388,0.012284,0.015893,0.008841,0.00000,0.004687,0.004386,...,0.011234,0.014398,6.028670,0.009638,0.239152,0.047557,0.016392,0.007013,0.010876,0.005726
A2M,692.432000,683.165000,157.053000,1643.760000,1279.810000,1311.630000,839.501000,52.79900,45.778700,66.294100,...,79.867700,119.502000,278.726000,286.536000,176.050000,65.231800,457.219000,637.325000,259.814000,1.261570
A2ML1,0.115811,0.099195,0.126896,0.173203,0.170873,0.135677,0.416320,1.36667,1.452340,1.813800,...,85.963400,81.979000,0.121853,0.132588,0.129857,8.122970,0.145466,0.330162,234.742000,0.046193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,264.034000,235.912000,70.748400,570.868000,416.840000,508.046000,400.830000,26.52500,41.943100,25.347800,...,113.264000,143.671000,145.308000,239.367000,87.356600,153.602000,132.050000,743.002000,248.278000,685.078000
ZZEF1,19.892300,14.695600,15.821000,23.267400,20.339000,24.895800,21.470000,4.96975,6.044240,6.155440,...,20.582600,22.154200,37.807800,25.619800,16.345900,20.717600,25.892000,25.629500,17.664600,9.162160
ZZZ3,10.986500,8.607600,9.063640,9.843830,11.112600,12.894200,12.289500,4.04317,3.985290,4.046860,...,10.813400,11.453800,7.666690,7.493900,6.135160,6.293300,14.359900,15.325900,11.913300,0.844963


In [18]:
LNC = luad_data.LncRNA.get_annotations()
MIR = luad_data.MicroRNA.get_annotations()
GE = luad_data.MessengerRNA.get_annotations()
print(GE.columns)
print(MIR.columns)
print(LNC.columns)

Index(['transcript_id', 'RNA type', 'go_id', 'Rfams', 'gene_id',
       'transcript_name', 'chromosome_name', 'transcript_start',
       'transcript_end', 'transcript_length', 'gene_biotype',
       'transcript_biotype', 'Transcript sequence', 'disease_associations'],
      dtype='object')
Index(['RNAcentral id', 'mirbase id', 'database', 'transcript_id', 'RNA type',
       'go_id', 'Rfams', 'transcript_name', 'chromosome_name',
       'transcript_start', 'transcript_end', 'transcript_length',
       'gene_biotype', 'transcript_biotype', 'Transcript sequence',
       'disease_associations'],
      dtype='object')
Index(['gene_name', 'feature', 'start', 'end', 'strand', 'tag', 'havana_gene',
       'transcript_id', 'transcript_name', 'chromosome_name',
       'transcript_start', 'transcript_end', 'transcript_length', 'Rfams',
       'go_id', 'gene_biotype', 'transcript_biotype', 'Transcript sequence',
       'disease_associations'],
      dtype='object')


In [19]:
# import pickle
# with open('moge/data/luad_data_openomics.pickle', 'wb') as file:
# #     luad_data = pickle.load(file)
#     pickle.dump(luad_data, file)

# Add tissue median gene expressions

In [ ]:
gene_exp_medians = pd.read_csv("/data/datasets/Bioinformatics_ExternalData/GTEx/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct", 
                               sep='\t', header=1, skiprows=1)
gene_exp_medians["gene_id"] = gene_exp_medians["gene_id"].str.replace("[.].*", "")
gene_exp_medians.set_index("gene_id", inplace=True)
gene_exp_medians.drop("Description", axis=1, inplace=True)

In [ ]:
transcript_exp_medians = pd.read_csv("/data/datasets/Bioinformatics_ExternalData/GTEx/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_max_tpm.tsv",
                                     sep='\t')
transcript_exp_medians["gene_id"] = transcript_exp_medians["gene_id"].str.replace("[.].*", "")
transcript_exp_medians.set_index("gene_id", inplace=True)
transcript_exp_medians
# # transcript_exp_medians = transcript_exp_medians.T
# transcript_exp_medians.index = transcript_exp_medians.index.str.replace("[.].*", "")

In [ ]:
mir_gene_id_to_name = pd.Series(MIR["mirbase id"].values, index=MIR["transcript_id"]).to_dict()
lnc_gene_id_to_name = pd.Series(LNC.index.values, index=LNC.index).to_dict()

In [ ]:
GE_tissue_exp = gene_exp_medians[gene_exp_medians.index.isin(GE.index)]
GE_tissue_exp = GE_tissue_exp[~GE_tissue_exp.index.duplicated(keep="first")]
GE_tissue_exp.shape

In [ ]:
LNC_tissue_exp = transcript_exp_medians[transcript_exp_medians.index.isin(LNC.index)]
# LNC_tissue_exp.index = LNC_tissue_exp.index.map(lnc_gene_id_to_name)
LNC_tissue_exp.shape

In [ ]:
pd.concat([GE_tissue_exp, LNC_tissue_exp], join="inner", copy=True)

In [ ]:
MIR_tissue_exp = transcript_exp_medians[transcript_exp_medians.index.isin(MIR.index)]
MIR_tissue_exp["transcript_id"] = MIR_tissue_exp.index
# MIR_tissue_exp["transcript_id"].replace(mir_gene_id_to_name, inplace=True)
MIR_tissue_exp.set_index("transcript_id", inplace=True)
# MIR_tissue_exp.drop("transcript_id", axis=1, inplace=True)
MIR_tissue_exp.shape

# Heterogeneous Network

In [3]:
import pickle
with open('moge/data/luad_data_openomics.pickle', 'rb') as file:
    luad_data = pickle.load(file)
#     pickle.dump(luad_data, file)

In [ ]:
from moge.network.heterogeneous_network import HeterogeneousNetwork

network = HeterogeneousNetwork(modalities=["MicroRNA", "MessengerRNA", "LncRNA"], multi_omics_data=luad_data)

## Write node labels to file

In [ ]:
from moge.network.file_io import write_node_labels_to_file

# write_node_labels_to_file("moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_node_labels_family.tsv", luad_data, 
#                           label_cols=["family"],
#                           modalities=["MIR", "GE", "LNC"], 
#                           get_dummies=False)

# write_node_labels_to_file("moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_node_labels_disease_dummies.tsv", luad_data, 
#                           label_cols=["Disease association"],
#                           modalities=["MIR", "GE", "LNC"], remove_na=True, sep="\t",
#                           get_dummies=True)

# Add Attribute Affinity Positive Edges

In [ ]:
affinities_GE = network.add_edges_from_nodes_similarity(modality="GE", node_list=network.nodes["GE"], 
    similarity_threshold=0.80, dissimilarity_threshold=0.01,
    negative_sampling_ratio=5.0, nanmean=True,
    features=["GO Terms", "Disease association"],
    weights=[1, 1],
    compute_correlation=False)

In [ ]:
affinities_GE = network.add_edges_from_nodes_similarity(modality="GE", node_list=network.nodes["GE"], 
    similarity_threshold=0.85, dissimilarity_threshold=0.01,
    negative_sampling_ratio=5.0, nanmean=False,
    features=["locus_type", "gene_family_id", "location"], 
    weights=[0.5, 1, 0.5, 1],
    compute_correlation=True, tissue_expression=GE_tissue_exp)

In [ ]:
affinities_MIR = network.add_edges_from_nodes_similarity(modality="MIR", node_list=network.nodes["MIR"], 
    similarity_threshold=0.70, dissimilarity_threshold=0.01,
    negative_sampling_ratio=5.0, nanmean=True,
    features=["GO Terms", "Disease association"], 
    compute_correlation=False)

In [ ]:
affinities_MIR = network.add_edges_from_nodes_similarity(modality="MIR", node_list=network.nodes["MIR"], 
    similarity_threshold=0.70, dissimilarity_threshold=0.01,
    negative_sampling_ratio=5.0, nanmean=True,
    features=["Family", "Rfams"], 
    compute_correlation=True, tissue_expression=MIR_tissue_exp)

In [ ]:
affinities_LNC = network.add_edges_from_nodes_similarity(modality="LNC", node_list=network.nodes["LNC"], 
    similarity_threshold=0.90, dissimilarity_threshold=0.1,
    negative_sampling_ratio=5.0, 
    nanmean=False,
    features=["locus_type", "Transcript type", "tag", "Strand", "Chromosome"],
    weights=[1, 1, 0.5, 0.5, 1, 1],
    compute_correlation=True, tissue_expression=LNC_tissue_exp)

In [ ]:
affinities_LNC = network.add_edges_from_nodes_similarity(modality="LNC", node_list=network.nodes["LNC"], 
    similarity_threshold=0.70, dissimilarity_threshold=0.01,
    negative_sampling_ratio=5.0, nanmean=True,
    features=["GO Terms", "Family", "Disease association"], 
    compute_correlation=False)

In [ ]:
matrix_heatmap(network.get_adjacency_matrix(edge_types=["u"], node_list=network.nodes["GE"]).todense(),
              figsize=(7,7))

In [ ]:
matrix_heatmap(network.get_adjacency_matrix(edge_types=["u"], node_list=network.nodes["MIR"]).todense(),
              figsize=(5,5))

In [ ]:
matrix_heatmap(network.get_adjacency_matrix(edge_types=["u"], node_list=network.nodes["LNC"]).todense(),
              figsize=(7,7))

In [ ]:
# IMPORT Affinity Edgelist
network.import_edgelist_file(
#     file="moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/lmn_n70_m70_l70-70_TissueExp_GO_Rfams_Disease_Family_GO_affinity.edgelist", 
    file="moge/data/LMN_future_recall/TRAIN/Interactions_Only/GE/lmn_train.BioGRID.interactions.edgelist", 
    is_directed=True)

network.import_edgelist_file(
    file="moge/data/LMN_future_recall/TRAIN/Interactions_Only/MIR/lmn_train.miRTarBase.interactions.edgelist", 
    is_directed=True)

network.import_edgelist_file(
    file="moge/data/LMN_future_recall/TRAIN/Interactions_Only/LNC/lmn_train.lncBase.interactions.edgelist", 
    is_directed=True)

network.import_edgelist_file(
    file="moge/data/LMN_future_recall/TRAIN/Interactions_Only/LNC/lmn_train.lncrna2target.interactions.edgelist", 
    is_directed=True)

In [ ]:
# WRITE Affinity Edgelist
# nx.write_edgelist(network.get_subgraph(["MIR", "GE", "LNC"], edge_type="u"), 
#                   "moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/lmn_n70_m70_l70-70_TissueExp_GO_Rfams_Disease_Family_GO_affinity.edgelist", 
#                   data=True)

# Add Attribute Affinity NEGATIVE Edges between cross-modals

In [ ]:
u_n_size = 10000
network.add_sampled_negative_edges(u_n_size, modalities=["GE", "MIR"])
network.add_sampled_negative_edges(u_n_size*2, modalities=["GE", "LNC"])
network.add_sampled_negative_edges(u_n_size, modalities=["LNC", "MIR"])

# miRTarBase

In [ ]:
mirtarbase_new = luad_data.MIR.get_miRTarBase_miRNA_target_interaction(use_latest=True, data=False, rename_dict=case_relabel)
mirtarbase_old = luad_data.MIR.get_miRTarBase_miRNA_target_interaction(use_latest=False, data=False, rename_dict=case_relabel)
mirtarbase_diff = list(set(mirtarbase_new) - set(mirtarbase_old))
len(mirtarbase_diff)

In [ ]:
# Adds miRNA-target interaction network
network.add_directed_edges_from_edgelist(edgelist=luad_data.MIR.get_miRTarBase_miRNA_target_interaction(use_latest=False),
#                                                                                                        rename_dict=case_relabel), 
                                        modalities=["MIR", "GE"], correlation_weights=False, threshold=0.20,
                                        database="miRTarBase")

# StarBase mRNA-RNA

In [ ]:
starbase_new = luad_data.GE.get_starBase_RNA_RNA_interactions(min_expNum=1, data=False)
starbase_old = luad_data.GE.get_starBase_RNA_RNA_interactions(min_expNum=2, data=False)
starbase_diff = list(set(starbase_new) - set(starbase_old))
len(starbase_diff)

In [ ]:
# Adds Gene Regulatory Network edges
network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_starBase_lncRNA_RNA_interactions(),
                                        modalities=["LNC", "GE"], correlation_weights=False, threshold=0.2,
                                        database="starBase")

# BioGRID

In [ ]:
biogrid_new = luad_data.GE.get_BioGRID_GRN_edgelist(data=False, rename_dict=case_relabel,
                biogrid_interactions_file_path='/data/datasets/Bioinformatics_ExternalData/BioGRID/BIOGRID-ALL-3.5.169.tab2.txt')
biogrid_old = luad_data.GE.get_BioGRID_GRN_edgelist(data=False, rename_dict=case_relabel,
                biogrid_interactions_file_path='/data/datasets/Bioinformatics_ExternalData/BioGRID/BIOGRID-ALL-3.4.162.tab2.txt')
biogrid_diff = list(set(biogrid_new) - set(biogrid_old))
len(biogrid_diff)

In [ ]:
# Adds Gene Regulatory Network edges
network.add_directed_edges_from_edgelist(edgelist=luad_data.GE.get_BioGRID_GRN_edgelist(biogrid_interactions_file_path='/data/datasets/Bioinformatics_ExternalData/BioGRID/BIOGRID-ALL-3.4.162.tab2.txt'),
#                                                                                        rename_dict=case_relabel),
                                        modalities=["GE", "GE"], correlation_weights=False, threshold=0.2,
                                        database="BioGRID")

# NPInter

In [ ]:
from moge.network.heterogeneous_network import get_rename_dict
noncode_rename_dict = pd.Series(luad_data.LNC.noncode_func_df["Gene Name"].values,
     index=luad_data.LNC.noncode_func_df["NONCODE Gene ID"].str.split(".", expand=True)[0]).to_dict()
noncode_rename_dict = {k: noncode_rename_dict[k] for k in noncode_rename_dict if type(noncode_rename_dict[k])!=float}

lncbase_rename_dict = get_rename_dict(luad_data.LNC.get_genes_info(), "Gene ID")
lncbase_rename_dict.update(noncode_rename_dict)

In [ ]:
# lncbase_rename_dict.update(case_relabel)

In [ ]:
npinter_new = luad_data.LNC.get_NPInter_ncRNA_RNA_regulatory_interaction_edgelist(use_latest=True, data=False,
                                                                                 rename_dict=lncbase_rename_dict)
npinter_old = luad_data.LNC.get_NPInter_ncRNA_RNA_regulatory_interaction_edgelist(use_latest=False, data=False,
                                                                                 rename_dict=lncbase_rename_dict)
npinter_diff = list(set(npinter_new) - set(npinter_old))
len(npinter_diff)

In [ ]:
network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_NPInter_ncRNA_RNA_regulatory_interaction_edgelist(use_latest=False), 
#                                                                                                                       rename_dict=lncbase_rename_dict),
                                         modalities=["LNC", "GE"], correlation_weights=False, threshold=0.20,
                                        database="NPInter")

# lncBase

In [ ]:
lncbase_new = luad_data.LNC.get_lncBase_miRNA_lncRNA_predicted_interactions_edgelist(rename_dict=lncbase_rename_dict,
                                                                                     data=False)
lncbase_old = luad_data.LNC.get_lncBase_miRNA_lncRNA_interactions_edgelist(rename_dict=lncbase_rename_dict, 
                                                                           data=False)
lncbase_diff = list(set(lncbase_new) - set(lncbase_old))
print(len(lncbase_diff))

In [ ]:
# Adds miRNA-lncRNA interaction network
# network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_starBase_lncRNA_miRNA_interactions_edgelist(),
#                                         )
# network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_lncRNome_miRNA_binding_sites_edgelist(),
#                                          modalities=["LNC", "MIR"], correlation_weights=False, threshold=0.20,
#                                         database="lncRNome")

network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_lncBase_miRNA_lncRNA_interactions_edgelist(rename_dict=lncbase_rename_dict),
                                         modalities=["MIR", "LNC"], correlation_weights=False, threshold=0.20,
                                        database="lncBase")


# network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_LncReg_lncRNA_RNA_regulatory_interactions(),
#                                          modalities=["LNC", "GE"], correlation_weights=False, threshold=0.20,
#                                         database="LncReg")

# lncrna2target

In [ ]:
lnc2tar_low = luad_data.LNC.get_lncrna2target_low_throughput_interactions(data=False, rename_dict=lncbase_rename_dict)
lnc2tar_high = luad_data.LNC.get_lncrna2target_high_throughput_interactions(data=False, rename_dict=lncbase_rename_dict)
lnc2tar_diff = list(set(lnc2tar_high) - set(lnc2tar_low))
len(lnc2tar_diff)

In [ ]:
network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_lncrna2target_high_throughput_interactions(rename_dict=lncbase_rename_dict),
                                         modalities=["LNC", "GE"], correlation_weights=False, threshold=0.20,
                                        database="lncrna2target")

# network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_lncrna2target_low_throughput_interactions(rename_dict=lncbase_rename_dict),
#                                          modalities=["LNC", "GE"], correlation_weights=False, threshold=0.20,
#                                         database="lncrna2target")

# network.add_directed_edges_from_edgelist(edgelist=luad_data.LNC.get_lncRInter_interactions(),
#                                          modalities=["LNC", "GE"], correlation_weights=False, threshold=0.20,
#                                         database="lncRInter")

In [ ]:
npinter_val = {(u,v) for u,v,d in network.G.edges(data=True) if "database" in d and d["database"]=="NPInter"}
len(npinter_val)
npinter_train = {(u,v) for u,v,d in network.G.edges(data=True) if "database" in d and d["database"]=="NPInter"}
len(npinter_train)
len(lncbase_train & l2t_val)

# Filter the edges

In [ ]:
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_lncbase_mirtarbase_biogrid_npinter_lncrna2target.train_v2.pickle', 
          'rb') as file:
    network = pickle.load(file)

In [ ]:
edges_bunch = [(u,v,d) for u,v,d in network.G.edges(data=True) if ("database" in d and 
                                                                   d["database"]=="miRTarBase")]
print(len(edges_bunch))

In [ ]:
database_nodes = nx.DiGraph(incoming_graph_data=edges_bunch).nodes()
database_nodes = list(database_nodes)
len(database_nodes)

In [ ]:
edges_bunch = [(u,v,d) for u,v,d in network.G.edges(nbunch=database_nodes, data=True) \
               if d["type"] == 'u' or d["type"] == "u_n" or ("database" in d and d["database"]=="miRTarBase")]
print(len(edges_bunch))

In [ ]:
network.G.clear()
network.G.add_edges_from(edges_bunch)

# Filter the node_list

In [ ]:
node_pairs = {(node1, node2) for node1 in network.G.nodes for node2 in network.G.nodes if\
              node1!=node2 and node2.lower()==node1.lower()}
len(node_pairs)

In [ ]:
case_relabel = {}
for u,v in node_pairs:
    if v in network.genes_info.index:
        case_relabel[u] = v
    elif u in network.genes_info.index:
        case_relabel[v] = u
len(case_relabel)

In [ ]:
len([node for node in network.G.nodes if node not in network.genes_info.index])

In [ ]:
isolates = list(nx.isolates(network.G))
len(isolates)

In [ ]:
network.G.remove_nodes_from(isolates)

In [ ]:
network.nodes["MessengerRNA"] = [node for node in network.nodes["MessengerRNA"] if node in network.G.nodes()]
network.nodes["LncRNA"] = [node for node in network.nodes["LncRNA"] if node in network.G.nodes()]
network.nodes["MicroRNA"] = [node for node in network.nodes["MicroRNA"] if node in network.G.nodes()]

In [ ]:
network.G.number_of_nodes(), network.G.number_of_edges()

In [ ]:
len(network.node_list)

In [ ]:
# WRITE/READ FULL network
import pickle
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Only/LMN_lncbase_mirtarbase_biogrid_lncrna2target_openomics.train.pickle', 
          'wb') as file:
    pickle.dump(network, file)
#     network = pickle.load(file)

# Train test split

In [ ]:
network.split_train_test_nodes(node_list=network.node_list, verbose=True,)

In [ ]:
train_generator = network.get_train_generator()

In [ ]:
test_generator = network.get_test_generator()

## Network Info & Visualizations

In [ ]:
print(len(network.G.nodes()))
print(len(network.G.edges()))

In [ ]:
csr_un = network.get_adjacency_matrix(edge_types=["u_n"], node_list=network.node_list)
csr_u = network.get_adjacency_matrix(edge_types=["u"], node_list=network.node_list)
csr_d = network.get_adjacency_matrix(edge_types=["d"], node_list=network.node_list)

In [ ]:
matrix_heatmap(csr_un.toarray())

In [ ]:
matrix_heatmap(csr_u.toarray())

In [ ]:
matrix_heatmap(csr_d.toarray())